In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 11 18:06:00 2022

@author: Beau.Uriona
"""

import folium
import pandas as pd
import geopandas as gpd
from requests import Session

# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.conversions import query_to_pandas
from snowexsql.data import SiteData, PointData, LayerData, ImageData

# Get the snow ex sites
SNOWEX_DB_NAME = 'snow:hackweek@db.snowexdata.org/snowex'

engine, session = get_db(SNOWEX_DB_NAME)
qry = session.query(SiteData.site_name, SiteData.site_id, SiteData.latitude, SiteData.longitude).distinct()

df_snow_ex = query_to_pandas(qry, engine).drop_duplicates(subset=["site_name"])
gdf_snow_ex = gpd.GeoDataFrame(
    df_snow_ex, 
    geometry=gpd.points_from_xy(df_snow_ex.longitude, df_snow_ex.latitude),
    crs=4326
)
#gdf_snow_ex.rename(columns={"geom": "geometry"}, inplace=True)

print(f"The CRS of the Snow Ex metadata is - {gdf_snow_ex.crs}")
gdf_snow_ex_buffer = gdf_snow_ex.buffer(0.5)
snow_ex_bounds = gdf_snow_ex.to_crs(4326).total_bounds
print(f"The bounding box of the snoe Ex sites is - {snow_ex_bounds}")
snow_ex_map = gdf_snow_ex.to_crs(4326).explore(tooltip=[i for i in gdf_snow_ex.columns if not i.lower().startswith("geo")])
snow_ex_map = gdf_snow_ex_buffer.to_crs(4326).explore(m=snow_ex_map)

# Get SNOTEL Sites
AWDB_API_DOMAIN = "https://api.snowdata.info"
sntl_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=SNTL&format=geojson"
# Get the data and rename everything to match the snow ex columns
gdf_sntl = gpd.read_file(sntl_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
print(f"The CRS of the Snotel metadata is - {gdf_sntl.crs}, better change it")
gdf_sntl = gdf_sntl.to_crs(gdf_snow_ex.crs)
print(f"The CRS of the Snotel metadata is now - {gdf_sntl.crs}, all good!")
# Only keep the columns we care about... for now
gdf_sntl = gdf_sntl.drop(
    columns=[i for i in gdf_sntl.columns if i not in ["geometry", "site_id", "site_name"]]
)
gdf_sntl_buffer = gdf_sntl.buffer(0.5)
sntl_bounds = gdf_sntl.to_crs(4326).total_bounds
print(f"The bounding box of the Snotel sites is - {snow_ex_bounds}")
sntl_map = gdf_sntl.explore(tooltip=[i for i in gdf_snow_ex.columns if not i.lower().startswith("geo")])
sntl_map = gdf_sntl_buffer.explore(m=sntl_map)

/srv/conda/envs/notebook/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


The CRS of the Snow Ex metadata is - epsg:4326
The bounding box of the snoe Ex sites is - [-120.23981   35.88858 -105.5569    43.94735]


/tmp/ipykernel_1165/4282798129.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_snow_ex_buffer = gdf_snow_ex.buffer(0.5)


The CRS of the Snotel metadata is - epsg:4979, better change it
The CRS of the Snotel metadata is now - epsg:4326, all good!
The bounding box of the Snotel sites is - [-120.23981   35.88858 -105.5569    43.94735]


/tmp/ipykernel_1165/4282798129.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_sntl_buffer = gdf_sntl.buffer(0.5)


In [2]:
snow_ex_map

In [3]:
gdf_nearest = gpd.sjoin_nearest(gdf_snow_ex.to_crs(26912), gdf_sntl.to_crs(26912), distance_col="distance").set_geometry(col="geometry")
gdf_nearest

,site_name_left,site_id_left,latitude,longitude,geometry,index_right,site_name_right,site_id_right,distance
0,Boise River Basin,Mores Creek Summit,43.947350,-115.676660,POINT (124705.961 4876666.746),564,Mores Creek Summit,637:ID:SNTL,1915.132673
1,Niwot Ridge,Forest South,40.040320,-105.556900,POINT (964469.260 4446448.876),589,Niwot,663:CO:SNTL,1119.610649
2,East River,Open 4,38.926320,-106.979210,POINT (848588.603 4316292.362),320,Butte,380:CO:SNTL,4206.827323
3,Jemez River,BA Flux Tower,35.888580,-106.531800,POINT (903402.679 3980822.401),147,Garita Peak,1173:NM:SNTL,12987.162107
5,Grand Mesa,9N39,39.031005,-108.166217,POINT (745296.199 4324038.780),550,Mesa Lakes,622:CO:SNTL,9855.613958
14,Cameron Pass,Michigan River,40.518690,-105.891990,POINT (932791.351 4497881.144),483,Joe Wright,551:CO:SNTL,1621.456874
15,Mammoth Lakes,CUES,37.643280,-119.029080,POINT (-208922.750 4196687.560),764,Virginia Lakes Ridge,846:CA:SNTL,51301.471346
25,Fraser Experimental Forest,JPL 1,39.905560,-105.882850,POINT (937503.253 4429826.918),155,Fool Creek,1186:CO:SNTL,4306.862987
68,Senator Beck,Swamp Angel,37.907100,-107.711180,POINT (789152.774 4200609.231),636,Red Mountain Pass,713:CO:SNTL,1729.131711
112,Little Cottonwood Canyon,Atwater,40.591250,-111.637590,POINT (446047.835 4493578.492),232,Atwater,1308:UT:SNTL,13.584986


In [4]:
nearest_map = gdf_nearest.explore(
    "distance", 
    tooltip=[i for i in gdf_nearest.columns if not i.lower().startswith("geo")],
    marker_kwds={"radius": 10},
    vmin=0,
)
nearest_map

In [6]:

import os
from os import path, makedirs

curr_path = os.getcwd()
output_dir = path.join(curr_path, "output")
makedirs(output_dir, exist_ok=True)
shp_dir = path.join(output_dir, "shp")
makedirs(shp_dir, exist_ok=True)
geojson_dir = path.join(output_dir, "geojson")
makedirs(geojson_dir, exist_ok=True)

snow_ex_dir = path.join(shp_dir, "snow-ex")
makedirs(snow_ex_dir, exist_ok=True)
gdf_snow_ex.to_crs(4326, inplace=True)
gdf_snow_ex.set_geometry(col="geometry", inplace=True)
gdf_snow_ex.to_file(path.join(snow_ex_dir, "snow_ex_stations.shp"), crs=4326)
gdf_snow_ex.to_file(path.join(geojson_dir, "snow_ex_stations.json"), crs=4326, driver='GeoJSON')
                    
sntl_dir = path.join(shp_dir, "sntl")
makedirs(sntl_dir, exist_ok=True)
gdf_sntl.to_crs(4326, inplace=True)
gdf_sntl.to_file(path.join(sntl_dir, "sntl_stations.shp"), crs=4326)
gdf_sntl.to_file(path.join(geojson_dir, "sntl_stations.json"), crs=4326, driver='GeoJSON')
                 
nearest_dir = path.join(shp_dir, "nearest")
makedirs(nearest_dir, exist_ok=True)
gdf_nearest.to_file(path.join(nearest_dir,  "nearest_stations.shp"), crs=4326)
gdf_nearest.to_file(path.join(geojson_dir,  "nearest_stations.json"), crs=4326, driver='GeoJSON')

/tmp/ipykernel_1165/719509236.py:27: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nearest.to_file(path.join(nearest_dir,  "nearest_stations.shp"), crs=4326)
